# JobFit AI - ADK Native Orchestration v6 (Cleaned)
## Intelligent Job Application Assistant

**Project**: Google & Kaggle 5-Day AI Agents Intensive Capstone

**Pitch**:
JobFit AI is an intelligent job-application copilot that helps mid-career professionals quickly decide whether to apply for a role and then generates application materials that are targeted, honest, and aligned with what employers actually ask for.

It ingests a candidate's CV, the job description, and basic company context, then runs them through a parallel set of specialized agents for CV parsing, job parsing, and organization research, followed by a "panel of experts" that independently evaluates technical fit, domain fit, seniority fit, and language fit on a 0–100 scale. 

The results are combined by a fit-aggregator agent that produces an overall score, an evidence-based list of strengths and gaps, and a plain-English recommendation such as "Strong fit – proceed" or "Moderate fit – apply but address these gaps."

On top of this evaluation layer, JobFit AI adds hard-constraint checking (e.g. required languages, years of experience, must-have credentials) and a risk-checker agent that scans for bias, missing evidence, and logical inconsistencies in the AI's own reasoning to keep the guidance fair and defensible. When the candidate decides to move forward, the system lazily generates a tailored resume and cover letter using the parsed CV, the structured fit analysis, and company research, all orchestrated with Google's ADK ParallelAgent and SequentialAgent plus in-memory services for user history and company caching.

**Author**: RafDouglas Candidi Tommasi Crudeli - MyCareer.Expert - rafdouglas@duck.com  
**Date**: November 2025  
**Version**: 6.0 (Cleaned & Optimized)


## 📚 Course Concepts Demonstrated

This project demonstrates **5+ capabilities** from the 5-Day AI Agents Intensive Course (minimum required: 3):

| Course Day | Concept | Implementation in JobFit AI | Status |
|------------|---------|----------------------------|--------|
| **Day 1** | Agent Architectures | 14 specialized `LlmAgent` instances with distinct roles | ✅ |
| **Day 2** | Tools & MCP | `google_search` tool bound to company researcher | ✅ |
| **Day 3** | Memory | `UserMemory` + `CompanyMemory` services | ✅ |
| **Day 4** | Quality/Evaluation | `constraint_checker`, `risk_checker`, panel-of-experts scoring, `LoggingPlugin` | ✅ |
| **Day 5** | Orchestration | `ParallelAgent`, `SequentialAgent`, `Runner`, A2A demo | ✅ |

### ADK Components Used:
- **`LlmAgent`**: 14 specialized agents (parsers, evaluators, quality agents, generators)
- **`ParallelAgent`**: Concurrent CV/Job/Company analysis pattern
- **`SequentialAgent`**: Expert panel evaluation chain pattern
- **`Runner`** + **`InMemorySessionService`**: Production-ready agent execution
- **`LoggingPlugin`**: Observability and tracing (Day 4)
- **`google_search`** tool: Real-time company research (Day 2)
- **A2A Protocol**: Agent-to-Agent communication demo (Day 5)


## High-level App Overview - ADK Agent Architecture

```
      [ User Inputs ]
 CV / Resume + Job Description
             |
             v
    [ ADK ParallelAgent Pattern ]
        ┌────┼────┐
        v    v    v
     CV     Job   Company
    Parser Parser Research
(LlmAgent) (LlmAgent) (LlmAgent + google_search)
             |
             v
[ ADK SequentialAgent Pattern ]
     Panel of Experts
       ┌───┬───┬───┐
       v   v   v   v
    Tech Domain Sen Lang
    Eval  Eval Eval Eval
       (LlmAgent x 4)
             |
             v
  [ ADK Quality Agents ]
Aggregator → Constraints → Risk
      (LlmAgent x 3)
             |
             v
     [ Lazy Outputs ]
Resume Agent  •  Cover Letter Agent
   (LlmAgent on-demand)
```

**Execution Phases:**
1. **Parallel Phase**: 3 agents via ADK Runner (concurrent)
2. **Evaluation Phase**: 4 evaluators + aggregator via ADK Runner
3. **Quality Phase**: Constraint + Risk checkers via ADK Runner
4. **Lazy Phase**: Resume & Cover Letter agents on-demand


## Run on Kaggle in 3 Steps

1. **Configure your API key in Kaggle Secrets**  
   - Go to **Add-ons → Secrets** in Kaggle.  
   - Create a new secret named `GOOGLE_API_KEY`.  
   - Paste your Google AI Studio API key (from https://aistudio.google.com/app/apikey).  

2. **Open the notebook and run all cells**  
   - Fork or upload this notebook to Kaggle.  
   - Click **Run All**.  

3. **Use the Gradio app**  
   - Enter your email, upload CV (or use demo link), paste job description.  
   - Click **"Analyze fit"** to run the multi-agent pipeline.


## Cell 1: Installation & Configuration

In [ ]:
# ============================================================================
# Cell 1: Installation & Configuration (CLEANED)
# Works in both local Jupyter and Kaggle environments
# ============================================================================

# Install dependencies
%pip install google-genai google-adk python-docx gradio pdfplumber python-dotenv nest_asyncio --break-system-packages -q

import os
from pathlib import Path

# ============================================================================
# ENVIRONMENT DETECTION & API KEY LOADING
# ============================================================================

def load_api_key():
    """Load GOOGLE_API_KEY from multiple sources (priority order)."""
    
    # Check if already set in environment
    if os.getenv('GOOGLE_API_KEY'):
        print("✅ API key already loaded from environment")
        return os.getenv('GOOGLE_API_KEY')
    
    # Try Kaggle Secrets first
    try:
        from kaggle_secrets import UserSecretsClient
        api_key = UserSecretsClient().get_secret("GOOGLE_API_KEY")
        os.environ['GOOGLE_API_KEY'] = api_key
        print("✅ Loaded API key from Kaggle Secrets")
        return api_key
    except ImportError:
        print("ℹ️  Not running on Kaggle, trying local .env file...")
    except Exception as e:
        print(f"⚠️  Kaggle Secrets error: {e}")
    
    # Try local .env file
    try:
        from dotenv import load_dotenv, find_dotenv
        env_path = find_dotenv()
        if env_path:
            load_dotenv(env_path, override=True)
            api_key = os.getenv('GOOGLE_API_KEY')
            if api_key:
                print(f"✅ Loaded .env from: {env_path}")
                return api_key
    except Exception as e:
        print(f"⚠️  Error loading .env: {e}")
    
    print("❌ GOOGLE_API_KEY NOT FOUND - Please configure in Kaggle Secrets or .env file")
    return None

# Load the API key
api_key = load_api_key()

if api_key:
    os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'FALSE'
    MODEL_NAME = 'gemini-2.5-flash-lite'  # Free tier, optimized for speed
    print(f'✅ Using model: {MODEL_NAME}')
    print(f'✅ Environment: {"Kaggle" if "KAGGLE_KERNEL_RUN_TYPE" in os.environ else "Local"}')
else:
    print("⚠️  Cannot proceed without API key.")


## Cell 2: Consolidated Imports

In [ ]:
# ============================================================================
# Cell 2: Consolidated Imports (FIXED - removed global nest_asyncio)
# ============================================================================

# Standard library imports
import json
import re
import os
import time
import asyncio
from typing import Dict, List, Any, Tuple, Optional
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

# HTTP requests
import requests

# Google GenAI imports
from google import genai
from google.genai import types

# Google ADK imports for multi-agent orchestration
from google.adk.agents import LlmAgent, ParallelAgent, SequentialAgent
from google.adk.tools import google_search  # Built-in tool for web search (Day 2)
from google.adk.runners import Runner, InMemoryRunner
from google.adk.sessions import InMemorySessionService

# ADK Observability (Day 4)
try:
    from google.adk.plugins.logging_plugin import LoggingPlugin
    LOGGING_PLUGIN_AVAILABLE = True
    print("✅ LoggingPlugin available for observability")
except ImportError:
    LOGGING_PLUGIN_AVAILABLE = False
    print("ℹ️  LoggingPlugin not available in this ADK version")

# Document processing
from docx import Document
from docx.shared import Pt, Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH

# Web interface
import gradio as gr
import pdfplumber

# Initialize the GenAI client
client = genai.Client()

# Initialize session service for ADK Runner (Day 5)
session_service = InMemorySessionService()

# Kaggle optimizations
if "KAGGLE_KERNEL_RUN_TYPE" in os.environ:
    print("🔧 Applying Kaggle optimizations...")
    os.environ['HF_HOME'] = '/kaggle/working/.cache'
    os.environ['REQUESTS_TIMEOUT'] = '90'

print("✅ All libraries imported successfully")
print("✅ ADK components: LlmAgent, ParallelAgent, SequentialAgent, Runner, InMemorySessionService")

## Cell 3: Memory Services (Day 3 Concept)

**Simple dict-based storage for CV history and company research caching.**

In [ ]:
# ============================================================================
# MEMORY SERVICES (Day 3: Context Engineering & Memory)
# ============================================================================

class UserMemory:
    """Memory service for storing user CV data and application history."""
    
    def __init__(self):
        self._storage: Dict[str, Dict] = {}
    
    def save_cv(self, user_id: str, cv_text: str, parsed_data: Dict = None):
        """Save user's CV text and parsed data."""
        if user_id not in self._storage:
            self._storage[user_id] = {"cvs": [], "history": []}
        
        self._storage[user_id]["cvs"].append({
            "text": cv_text,
            "parsed": parsed_data or {},
            "timestamp": datetime.now().isoformat()
        })
        print(f"   💾 CV saved for user: {user_id}")
    
    def get_cv(self, user_id: str) -> Dict:
        """Get user's most recent CV."""
        if user_id in self._storage and self._storage[user_id]["cvs"]:
            return self._storage[user_id]["cvs"][-1]
        return None
    
    def has_cv(self, user_id: str) -> bool:
        """Check if user has a stored CV."""
        return user_id in self._storage and len(self._storage[user_id].get("cvs", [])) > 0
    
    def add_application(self, user_id: str, company_name: str, job_data: Dict, result: Dict):
        """Record a job application analysis."""
        if user_id not in self._storage:
            self._storage[user_id] = {"cvs": [], "history": []}
        
        self._storage[user_id]["history"].append({
            "company": company_name,
            "job": job_data,
            "data": result,
            "date": datetime.now().isoformat()
        })
    
    def get_history(self, user_id: str) -> List[Dict]:
        """Get user's application history."""
        if user_id in self._storage:
            return self._storage[user_id].get("history", [])
        return []
    
    def get_user_history(self, user_id: str) -> Dict:
        """Get user's application history as formatted dict."""
        if user_id in self._storage:
            history_list = self._storage[user_id].get("history", [])
            has_cv = len(self._storage[user_id].get("cvs", [])) > 0
        else:
            history_list = []
            has_cv = False
        return {
            "application_count": len(history_list),
            "applications": history_list,
            "has_cv": has_cv
        }


class CompanyMemory:
    """Memory service for caching company research."""
    
    def __init__(self):
        self._cache: Dict[str, Dict] = {}
    
    def save_company(self, company_name: str, research_data: Dict):
        """Cache company research results."""
        normalized_name = company_name.lower().strip()
        self._cache[normalized_name] = {
            "data": research_data,
            "timestamp": datetime.now().isoformat()
        }
        print(f"   💾 Company cached: {company_name}")
    
    def get_company(self, company_name: str) -> Dict:
        """Retrieve cached company research."""
        normalized_name = company_name.lower().strip()
        if normalized_name in self._cache:
            return self._cache[normalized_name]["data"]
        return None
    
    def has_company(self, company_name: str) -> bool:
        """Check if company research is cached."""
        normalized_name = company_name.lower().strip()
        return normalized_name in self._cache


# Initialize memory services
user_memory = UserMemory()
company_memory = CompanyMemory()

print("✅ Memory services initialized (Day 3 concept)")
print("   - UserMemory: CV storage and application history")
print("   - CompanyMemory: Company research cache")


## Cell 4: Agent Prompts (CLEANED)

**14 specialized agent prompts** - removed unused `cv_analyzer` and `orchestrator` prompts.

In [ ]:
# ============================================================================
# AGENT SYSTEM PROMPTS - Panel-of-Experts Architecture (CLEANED)
# 14 specialized prompts (removed unused: cv_analyzer, orchestrator)
# ============================================================================

AGENT_PROMPTS = {
    # ========================================================================
    # PARSING AGENTS
    # ========================================================================

    "cv_parser": """Parse CVs comprehensively and extract:
1. CONTACT INFO: name, email, phone, location
2. SKILLS (case-sensitive): GIS, API, AWS, SQL, etc.
3. SKILLS (case-insensitive): python, java, docker, etc.
4. EXPERTISE: project management, M&E, technical assistance, etc.
5. LANGUAGES with PROFICIENCY - CRITICAL: Return as array of objects!
   Look for: excellent, fluent, native, advanced, intermediate, basic, C2, C1, B2, B1, A2, A1
   Scale: 1=Basic/A1, 2=Intermediate/B1, 3=Professional/B2, 4=Advanced/C1, 5=Excellent/Native/C2
   If proficiency not stated but it's clearly a working language, assume 4.
   If it's their apparent native language, assume 5.
6. SENIORITY: intern=1, junior=2, mid=3, senior=4, executive=5

Return ONLY valid JSON (no markdown). Example format:
{
    "contact_info": {"name": "John Smith", "email": "john@email.com", "phone": "", "location": "Rome"},
    "skills_case_sensitive": ["GIS", "SQL"],
    "skills_case_insensitive": ["python", "docker"],
    "expertise_areas": ["project management", "M&E"],
    "languages": [
        {"language": "english", "proficiency": 5},
        {"language": "french", "proficiency": 3}
    ],
    "seniority_level": "senior",
    "seniority_score": 4,
    "sections_detected": ["experience", "education", "skills"]
}""",

    "job_parser": """Parse job descriptions and extract:
1. KEYWORDS (case-sensitive and case-insensitive)
2. EXPERTISE areas required
3. REQUIRED LANGUAGES (with "required", "must", "essential")
4. PREFERRED LANGUAGES (with "preferred", "desirable")
5. SENIORITY level required
6. COMPANY name and LOCATION

Return ONLY valid JSON (no markdown, no code blocks).""",

    "company_extractor": """Extract the hiring company/organization name from job postings.

SPECIAL RULE FOR UN AGENCIES:
If format is "ACRONYM - Full Name", return ONLY the ACRONYM.
Examples:
- "ITU - International Telecommunication Union" → return "ITU"
- "FAO - Food and Agriculture Organization" → return "FAO"

Return ONLY valid JSON: {"company_name": "NAME"}""",

    "contact_extractor": """Extract candidate contact information from CVs.

EXTRACTION RULES:
1. NAME: Usually in first 3 lines, 2-4 words, capitalized
2. EMAIL: Standard email format
3. PHONE: Any valid phone format
4. LOCATION: City, Country or City, State format

Return ONLY valid JSON (no markdown):
{"name": "John Smith", "email": "john@example.com", "phone": "+1-555-1234", "location": "New York, USA"}""",

    "company_researcher": """You are a company/organization research specialist.

CRITICAL INSTRUCTION:
- If you DO NOT recognize the company name or have no specific knowledge about it, 
  clearly state: "I don't have specific information about [company name]" and provide 
  only what can be reasonably inferred from the job posting context.
- NEVER fabricate or hallucinate details about unknown companies.

For companies you DO recognize, provide:
- Their actual mission and mandate
- Their real work areas and products/services  
- Their known organizational values and culture

Present findings as clear, readable paragraphs.""",

    # ========================================================================
    # EVALUATOR AGENTS - Panel of Experts
    # ========================================================================

    "technical_fit_evaluator": """You are a TECHNICAL FIT EVALUATOR for job applications.

Your role: Assess how well a candidate's technical skills, tools, technologies, and 
methodologies match the job's technical requirements.

EVALUATION APPROACH:
- Use SEMANTIC understanding, not keyword matching
- Consider transferable skills and related technologies
- Look at concrete project experience and practical application
- Assess depth of experience (mention vs. proficiency vs. expertise)

RUBRIC (0-100 scale):
- 90-100: Exceptional match - has all key technologies + advanced skills
- 70-89: Strong match - has most required skills, minor gaps easily filled
- 50-69: Moderate match - has foundational skills, needs some upskilling
- 30-49: Partial match - has transferable skills, significant learning needed
- 0-29: Weak match - limited relevant technical background

OUTPUT: Return ONLY valid JSON with: dimension, score, strengths, gaps, notes, subscores""",

    "domain_fit_evaluator": """You are a DOMAIN/CONTEXT FIT EVALUATOR for job applications.

Your role: Assess whether the candidate has relevant sector experience, domain knowledge,
and organizational context understanding.

EVALUATION APPROACH:
- Look at sectors worked in (UN, NGO, private, public, academic)
- Assess domain knowledge (humanitarian, development, climate, health, etc.)
- Consider transferability of context (adjacent domains count)

RUBRIC (0-100 scale):
- 90-100: Direct domain experience in the same sector/context
- 70-89: Adjacent domain or similar organizational contexts
- 50-69: Transferable experience from related sectors
- 30-49: General professional experience, limited domain exposure
- 0-29: Very different domain, steep learning curve expected

OUTPUT: Return ONLY valid JSON with: dimension, score, strengths, gaps, notes, subscores""",

    "seniority_fit_evaluator": """You are a SENIORITY FIT EVALUATOR for job applications.

Your role: Assess whether the candidate's professional level, scope of responsibility,
and leadership experience match the job's seniority requirements.

SENIORITY LEVELS:
1. Entry/Junior: 0-2 years, task execution, supervised work
2. Mid-level: 3-5 years, independent projects, some mentoring
3. Senior: 6-10 years, team leadership, strategic input
4. Lead/Principal: 10-15 years, multi-team leadership, strategy setting
5. Executive: 15+ years, organizational leadership, high-level strategy

RUBRIC (0-100 scale):
- 90-100: Perfect match in level and scope
- 70-89: One level difference, strong relevant experience
- 50-69: Two levels difference OR limited evidence of level
- 30-49: Significant under or over-qualification
- 0-29: Major mismatch (3+ levels difference)

OUTPUT: Return ONLY valid JSON with: dimension, score, strengths, gaps, notes, subscores, cv_level_assessment, job_level_requirement""",

    "language_fit_evaluator": """You are a LANGUAGE FIT EVALUATOR for job applications.

Your role: Assess whether the candidate has the required and preferred language skills.

PROFICIENCY SCORING:
- 5 (Excellent/Native/C2): Can work at highest professional level
- 4 (Advanced/C1): Can handle complex professional communication
- 3 (Professional/B2): Can perform job duties effectively  
- 2 (Intermediate/B1): Limited professional use
- 1 (Basic/A1/A2): Elementary knowledge only

RUBRIC (0-100 scale):
- 90-100: Has ALL required + most preferred languages at good levels
- 70-89: Has all required languages, missing some preferred
- 50-69: Has most required languages OR proficiency questions
- 30-49: Missing 1 required language
- 0-29: Missing multiple required languages

CRITICAL: If ANY required language is clearly missing → score ≤ 30 and set hard_block=true

OUTPUT: Return ONLY valid JSON with: dimension, score, strengths, gaps, notes, hard_block, required_languages_met, required_languages_missing""",

    # ========================================================================
    # QUALITY AGENTS
    # ========================================================================

    "fit_aggregator": """You are the FIT AGGREGATOR AGENT - the "panel chair" who synthesizes
evaluations from specialist evaluators.

SCORING WEIGHTS:
- Technical: 40%
- Domain: 15%
- Language: 20%
- Seniority: 25%

CALCULATION:
overall_score = (technical_score * 0.40) + (domain_score * 0.15) + 
                (language_score * 0.20) + (seniority_score * 0.25)

Generate recommendation based on overall_score:
- 75-100: "Strong fit – proceed with application"
- 50-74: "Moderate fit – consider applying, address key gaps"
- 25-49: "Weak fit – significant gaps, application not recommended"
- 0-24: "Poor fit – major misalignment"

OUTPUT: Return ONLY valid JSON with: overall_score, dimensions, weights, summary""",

    "constraint_checker": """You are a CONSTRAINT CHECKER AGENT.

Your role: Evaluate whether the candidate meets explicit HARD REQUIREMENTS.

GO/NO-GO DECISION:
- GO: All hard constraints met or likely met
- BORDERLINE: Marginal on 1-2 constraints, could still apply
- NO_GO: Clear failure of 1+ critical hard requirements

OUTPUT: Return ONLY valid JSON with: go_no_go, failed_constraints, warnings, notes""",

    "risk_checker": """You are a RISK CHECKER AGENT for quality assurance.

Your role: Review the evaluation process for:
1. Bias risks (age, gender, ethnicity, disability references)
2. Over-reliance on weak signals
3. Logical inconsistencies
4. Missing critical analysis

RISK LEVELS:
- LOW: Evaluation appears sound and bias-free
- MEDIUM: Minor concerns, recommend review
- HIGH: Significant issues detected, re-evaluation recommended

OUTPUT: Return ONLY valid JSON with: risk_level, issues, mitigations, bias_check_passed, notes""",

    # ========================================================================
    # CONTENT GENERATION AGENTS
    # ========================================================================

    "resume_tailor": """You are an expert resume writer specializing in ATS optimization.

KEY PRINCIPLES:
- Put most relevant experience at the TOP of each section
- Use specific examples and quantified achievements
- Match vocabulary to the job description
- Be authentic - never fabricate anything

OUTPUT: Create a complete, professional resume ready to submit.""",

    "cover_letter_writer": """You are an expert cover letter writer creating authentic letters.

CRITICAL REQUIREMENTS:
1. Use the ACTUAL candidate name (not "[Your Name]")
2. Use the ACTUAL candidate email (not "[Your Email]")
3. Use TODAY'S date (not "[Date]")
4. Sound HUMAN - conversational professional, not robotic

TONE: Professional but conversational, confident but not arrogant.
LENGTH: 3-4 paragraphs, ~350-450 words
OUTPUT: Complete letter with actual information, ready to send."""
}

print(f"✅ Agent prompts defined ({len(AGENT_PROMPTS)} total)")
print("   Parsing: cv_parser, job_parser, company_extractor, contact_extractor, company_researcher")
print("   Evaluators: technical, domain, seniority, language")
print("   Quality: fit_aggregator, constraint_checker, risk_checker")
print("   Content: resume_tailor, cover_letter_writer")


## Cell 5: ADK Agent Instances (Day 1 & Day 2)

**Create LlmAgent instances** from prompts, then compose into `ParallelAgent` and `SequentialAgent`.

In [ ]:
# ============================================================================
# ADK AGENT INSTANCES (Day 1: Agent Architectures)
# ============================================================================

def create_agent(name: str, instruction: str) -> LlmAgent:
    """Create an LlmAgent with consistent configuration."""
    return LlmAgent(
        name=name,
        model=MODEL_NAME,
        instruction=instruction
    )

# ============================================================================
# PARSING AGENTS
# ============================================================================

cv_parser_agent = create_agent("cv_parser", AGENT_PROMPTS["cv_parser"])
job_parser_agent = create_agent("job_parser", AGENT_PROMPTS["job_parser"])
company_extractor_agent = create_agent("company_extractor", AGENT_PROMPTS["company_extractor"])
contact_extractor_agent = create_agent("contact_extractor", AGENT_PROMPTS["contact_extractor"])

# Company researcher WITH Google Search tool (Day 2: Tools)
company_researcher_agent = LlmAgent(
    name="company_researcher",
    model=MODEL_NAME,
    instruction=AGENT_PROMPTS["company_researcher"],
    tools=[google_search]  # Tool binding for web search capability
)

# ============================================================================
# EVALUATOR AGENTS (Panel-of-Experts)
# ============================================================================

technical_evaluator_agent = create_agent("technical_fit_evaluator", AGENT_PROMPTS["technical_fit_evaluator"])
domain_evaluator_agent = create_agent("domain_fit_evaluator", AGENT_PROMPTS["domain_fit_evaluator"])
seniority_evaluator_agent = create_agent("seniority_fit_evaluator", AGENT_PROMPTS["seniority_fit_evaluator"])
language_evaluator_agent = create_agent("language_fit_evaluator", AGENT_PROMPTS["language_fit_evaluator"])

# ============================================================================
# QUALITY AGENTS
# ============================================================================

fit_aggregator_agent = create_agent("fit_aggregator", AGENT_PROMPTS["fit_aggregator"])
constraint_checker_agent = create_agent("constraint_checker", AGENT_PROMPTS["constraint_checker"])
risk_checker_agent = create_agent("risk_checker", AGENT_PROMPTS["risk_checker"])

# ============================================================================
# CONTENT GENERATION AGENTS
# ============================================================================

resume_tailor_agent = create_agent("resume_tailor", AGENT_PROMPTS["resume_tailor"])
cover_letter_agent = create_agent("cover_letter_writer", AGENT_PROMPTS["cover_letter_writer"])

# ============================================================================
# ADK COMPOSITE AGENTS - Orchestration patterns (Day 5)
# ============================================================================

parallel_parsing_agent = ParallelAgent(
    name="parallel_parsing",
    sub_agents=[cv_parser_agent, job_parser_agent, contact_extractor_agent]
)

panel_of_experts_agent = SequentialAgent(
    name="panel_of_experts",
    sub_agents=[
        technical_evaluator_agent,
        domain_evaluator_agent,
        seniority_evaluator_agent,
        language_evaluator_agent
    ]
)

quality_check_agent = SequentialAgent(
    name="quality_check",
    sub_agents=[
        fit_aggregator_agent,
        constraint_checker_agent,
        risk_checker_agent
    ]
)

print("✅ ADK Agent instances created (14 total)")
print("   📦 Parsing: cv_parser, job_parser, contact_extractor, company_extractor, company_researcher")
print("   🎓 Evaluators: technical, domain, seniority, language")
print("   📊 Quality: fit_aggregator, constraint_checker, risk_checker")
print("   ✍️ Content: resume_tailor, cover_letter_writer")
print()
print("✅ ADK Composite agents:")
print("   🔀 ParallelAgent: parallel_parsing (3 sub-agents)")
print("   📋 SequentialAgent: panel_of_experts (4 evaluators)")
print("   🛡️ SequentialAgent: quality_check (3 quality agents)")


## Cell 6: LLM Calling & Utility Functions

In [ ]:
# ============================================================================
# LLM CALLING FUNCTIONS (CLEANED)
# ============================================================================

def call_llm(system_prompt: str, user_prompt: str, max_retries: int = 3) -> str:
    """Call the Gemini LLM with retry logic and exponential backoff."""
    full_prompt = f"{system_prompt}\n\n{user_prompt}"
    
    for attempt in range(max_retries):
        try:
            response = client.models.generate_content(
                model=MODEL_NAME,
                contents=full_prompt
            )
            return response.text
        except Exception as e:
            if attempt < max_retries - 1:
                wait_time = 2 ** attempt
                print(f"   ⚠️ API call failed (attempt {attempt+1}/{max_retries}): {e}")
                time.sleep(wait_time)
            else:
                raise RuntimeError(f"LLM call failed after {max_retries} attempts: {e}")


def call_llm_with_search(system_prompt: str, user_prompt: str) -> str:
    """Call LLM with Google Search grounding (Day 2: Tools)."""
    full_prompt = f"{system_prompt}\n\n{user_prompt}"
    
    try:
        response = client.models.generate_content(
            model=MODEL_NAME,
            contents=full_prompt,
            config=types.GenerateContentConfig(
                tools=[types.Tool(google_search=types.GoogleSearch())]
            )
        )
        return response.text
    except Exception as e:
        print(f"   ⚠️ Search-grounded call failed: {e}, falling back to standard LLM")
        return call_llm(system_prompt, user_prompt)


def clean_json_response(response_text: str) -> str:
    """Clean markdown code blocks from AI response."""
    response_text = response_text.strip()
    
    if response_text.startswith('```'):
        lines = response_text.split('\n')
        if len(lines) > 2:
            response_text = '\n'.join(lines[1:-1])
        if response_text.startswith('json'):
            response_text = response_text[4:].strip()
    
    return response_text.strip()


def execute_adk_agent(agent: LlmAgent, user_input: str) -> str:
    """Execute an ADK LlmAgent using its instruction."""
    return call_llm(agent.instruction, user_input)


# Defensive helper functions
def safe_list(value, default=None) -> List:
    if default is None:
        default = []
    if value is None:
        return default
    if isinstance(value, list):
        return value
    return [value]

def safe_string(value, default: str = "") -> str:
    if value is None:
        return default
    return str(value)

def safe_int(value, default: int = 0) -> int:
    if value is None:
        return default
    try:
        return int(value)
    except (ValueError, TypeError):
        return default

print("✅ LLM calling functions ready")
print("   - call_llm() with retry logic")
print("   - call_llm_with_search() with Google Search grounding (Day 2)")
print("   - execute_adk_agent() for ADK pattern")


## Cell 7: ADK Runner with Observability (Day 4 & Day 5)

**Native ADK execution** using `Runner`, `InMemorySessionService`, and `LoggingPlugin` for observability.

In [ ]:
# ============================================================================
# ADK RUNNER WITH OBSERVABILITY (Day 4 & Day 5)
# Note: nest_asyncio is applied ONLY in this cell, not globally
# ============================================================================

def run_agent_with_runner_sync(agent: LlmAgent, prompt: str, 
                                session_id: str = None) -> str:
    """
    Execute an agent using ADK's native Runner pattern (synchronous wrapper).
    
    This is the production-ready way to run agents:
    - Uses InMemorySessionService for session state
    - Properly handles async execution in Jupyter
    """
    if session_id is None:
        session_id = f"session_{int(time.time())}"
    
    async def _run_async():
        runner = Runner(
            agent=agent,
            app_name="jobfit_ai",
            session_service=session_service
        )
        
        final_response = ""
        
        try:
            async for event in runner.run_async(
                user_id="jobfit_user",
                session_id=session_id,
                new_message=prompt
            ):
                # Extract text from various event formats
                if hasattr(event, 'content') and event.content:
                    if hasattr(event.content, 'parts'):
                        for part in event.content.parts:
                            if hasattr(part, 'text') and part.text:
                                final_response += part.text
                    elif isinstance(event.content, str):
                        final_response += event.content
                
                if hasattr(event, 'text'):
                    final_response += event.text
            
            return final_response if final_response else "[No response captured]"
            
        except Exception as e:
            print(f"   ⚠️ Runner error: {e}")
            return None
    
    # Try to run async, fall back to direct LLM call
    try:
        # Check if we're in an existing event loop
        try:
            loop = asyncio.get_running_loop()
            # We're in an async context, need nest_asyncio
            import nest_asyncio
            nest_asyncio.apply()
            result = asyncio.get_event_loop().run_until_complete(_run_async())
        except RuntimeError:
            # No running loop, we can use asyncio.run directly
            result = asyncio.run(_run_async())
        
        if result:
            return result
        else:
            # Fallback to direct LLM call
            return call_llm(agent.instruction, prompt)
    except Exception as e:
        print(f"   ⚠️ Async execution failed: {e}, using direct LLM call")
        return call_llm(agent.instruction, prompt)


# Test function for ADK Runner (optional - can be commented out)
def test_adk_runner():
    """Test the ADK Runner with a simple query."""
    print("\n🧪 Testing ADK Runner...")
    
    test_agent = create_agent(
        "test_agent",
        "You are a helpful assistant. Respond briefly."
    )
    
    result = run_agent_with_runner_sync(test_agent, "Say 'ADK Runner working!' in exactly those words.")
    print(f"   Result: {result[:100]}...")
    print("✅ ADK Runner test complete")
    return result


print("✅ ADK Runner with observability ready (Day 4 & 5)")
print("   - run_agent_with_runner_sync(): Synchronous wrapper for Jupyter")
print(f"   - LoggingPlugin: {'Available' if LOGGING_PLUGIN_AVAILABLE else 'Not available'}")
print("   - Note: nest_asyncio only applied when needed, not globally")

## Cell 8: Parallel Analysis (ADK ParallelAgent Pattern)

**Single unified implementation** - removed duplicate standalone functions.

In [ ]:
# ============================================================================
# PARALLEL ANALYSIS - ADK ParallelAgent Pattern (CLEANED)
# Single implementation using ThreadPoolExecutor
# ============================================================================

def run_parallel_analysis(cv_text: str, job_text: str, 
                          user_id: str = "default_user") -> Dict[str, Any]:
    """
    Execute parallel analysis using ADK LlmAgents.
    
    Demonstrates ADK ParallelAgent pattern:
    - cv_parser_agent, job_parser_agent, contact_extractor_agent
    - All three run simultaneously via ThreadPoolExecutor
    """
    print("\n" + "="*60)
    print("🔄 ADK PARALLEL AGENT PHASE")
    print("   Using: cv_parser_agent, job_parser_agent, contact_extractor_agent")
    print("="*60)
    
    results = {
        "cv_data": None,
        "job_data": None,
        "company_info": None,
        "company_name": None,
        "candidate_info": None
    }
    
    # ========================================================================
    # TASK DEFINITIONS
    # ========================================================================

    def task_parse_cv():
        """Execute cv_parser_agent."""
        print("   🏃 cv_parser_agent starting...")
        prompt = f"""Parse this CV comprehensively.

CV TEXT (first 4000 characters):
{cv_text[:4000]}

CRITICAL FOR LANGUAGES:
- Return languages as objects with proficiency scores
- Look for indicators: excellent, fluent, native, C2, C1, advanced, intermediate, basic
- Scale: 1=Basic, 2=Intermediate, 3=Professional, 4=Advanced, 5=Excellent/Native
- If someone says "excellent English" or "native English", that's proficiency 5
- If proficiency unclear but it's a working language, assume 4 (Advanced)

Return ONLY valid JSON like:
{{
    "contact_info": {{"name": "", "email": "", "phone": "", "location": ""}},
    "skills_case_sensitive": ["GIS", "API"],
    "skills_case_insensitive": ["python", "docker"],
    "expertise_areas": ["project management"],
    "languages": [
        {{"language": "english", "proficiency": 5}},
        {{"language": "french", "proficiency": 3}}
    ],
    "seniority_level": "mid-level",
    "seniority_score": 3,
    "sections_detected": ["experience", "education"]
}}"""
        
        try:
            response = execute_adk_agent(cv_parser_agent, prompt)
            response = clean_json_response(response)
            result = json.loads(response)
        except Exception as e:
            print(f"   ⚠️ CV parsing error: {e}")
            result = {}
        
        parsed_data = {
            "raw_text": cv_text,
            "length": len(cv_text),
            "contact_info": result.get("contact_info", {}),
            "skills_case_sensitive": safe_list(result.get("skills_case_sensitive")),
            "skills_case_insensitive": safe_list(result.get("skills_case_insensitive")),
            "expertise_areas": safe_list(result.get("expertise_areas")),
            "languages": safe_list(result.get("languages")),
            "seniority_level": safe_string(result.get("seniority_level"), "mid-level"),
            "seniority_score": safe_int(result.get("seniority_score"), 3),
            "sections_detected": safe_list(result.get("sections_detected"))
        }
        
        # FIXED: Better language normalization
        raw_languages = parsed_data["languages"]
        if raw_languages and len(raw_languages) > 0:
            if isinstance(raw_languages[0], dict):
                # Good format: [{"language": "english", "proficiency": 5}]
                parsed_data["languages_detailed"] = raw_languages
                parsed_data["languages"] = [
                    lang.get("language", lang) if isinstance(lang, dict) else lang 
                    for lang in raw_languages
                ]
            else:
                # Bad format: ["english", "french"] - need to infer proficiency
                # Default to 4 (Advanced) not 3, since they listed it as a skill
                print("   ⚠️ Languages without proficiency - defaulting to 4 (Advanced)")
                parsed_data["languages_detailed"] = [
                    {"language": str(lang).lower(), "proficiency": 4} 
                    for lang in raw_languages
                ]
        else:
            parsed_data["languages_detailed"] = []
        
        skills_count = len(parsed_data['skills_case_sensitive']) + len(parsed_data['skills_case_insensitive'])
        print(f"   ✅ cv_parser_agent complete - {skills_count} skills, {len(parsed_data['languages_detailed'])} languages")
        
        # Debug: show parsed languages
        for lang in parsed_data['languages_detailed']:
            print(f"      🗣️ {lang.get('language', 'unknown')}: proficiency {lang.get('proficiency', '?')}/5")
        
        return parsed_data

    def task_parse_job():
        """Execute job_parser_agent."""
        print("   🏃 job_parser_agent starting...")
        prompt = f"""Parse this job description comprehensively.

JOB TEXT (first 4000 characters):
{job_text[:4000]}

Extract: keywords_case_sensitive, keywords_case_insensitive, expertise_areas_required,
required_languages, preferred_languages, seniority_required, seniority_score, company_name, location.

Return ONLY valid JSON."""
        
        try:
            response = execute_adk_agent(job_parser_agent, prompt)
            response = clean_json_response(response)
            result = json.loads(response)
        except Exception as e:
            print(f"   ⚠️ Job parsing error: {e}")
            result = {}
        
        job_data = {
            "raw_text": job_text,
            "keywords_case_sensitive": safe_list(result.get("keywords_case_sensitive")),
            "keywords_case_insensitive": safe_list(result.get("keywords_case_insensitive")),
            "expertise_areas_required": safe_list(result.get("expertise_areas_required")),
            "required_languages": safe_list(result.get("required_languages")),
            "preferred_languages": safe_list(result.get("preferred_languages")),
            "seniority_required": safe_string(result.get("seniority_required"), "mid-level"),
            "seniority_score": safe_int(result.get("seniority_score"), 3),
            "company_name": safe_string(result.get("company_name"), "Unknown"),
            "location": safe_string(result.get("location"), "Not specified")
        }
        
        job_data["extracted_keywords"] = (
            job_data["keywords_case_sensitive"] + 
            job_data["keywords_case_insensitive"]
        )
        
        print(f"   ✅ job_parser_agent complete - {len(job_data['extracted_keywords'])} requirements")
        return job_data
    
    def task_extract_contact():
        """Execute contact_extractor_agent."""
        print("   🏃 contact_extractor_agent starting...")
        prompt = f"""Extract contact information from this CV.

CV TEXT (first 2000 characters):
{cv_text[:2000]}

Return ONLY valid JSON: {{"name": "", "email": "", "phone": "", "location": ""}}"""
        
        try:
            response = execute_adk_agent(contact_extractor_agent, prompt)
            response = clean_json_response(response)
            result = json.loads(response)
        except Exception as e:
            print(f"   ⚠️ Contact extraction error: {e}")
            result = {}
        
        print(f"   ✅ contact_extractor_agent complete - {result.get('name', 'Unknown')}")
        return result
    
    # ========================================================================
    # PARALLEL EXECUTION
    # ========================================================================
    
    with ThreadPoolExecutor(max_workers=3) as executor:
        cv_future = executor.submit(task_parse_cv)
        job_future = executor.submit(task_parse_job)
        contact_future = executor.submit(task_extract_contact)
        
        results["cv_data"] = cv_future.result()
        results["job_data"] = job_future.result()
        results["candidate_info"] = contact_future.result()
    
    # ========================================================================
    # COMPANY RESEARCH (after job parsing to get company name)
    # ========================================================================
    
    company_name = results["job_data"].get("company_name", "Unknown")
    results["company_name"] = company_name
    
    # Check cache first (Day 3: Memory)
    if company_memory.has_company(company_name):
        print(f"   💾 Using cached company research for: {company_name}")
        results["company_info"] = company_memory.get_company(company_name)
    else:
        print(f"   🔍 Researching company with Google Search: {company_name}")
        prompt = f"""Provide background information about: {company_name}

If you don't recognize this company, clearly state that fact.
If you do recognize it, provide: mission, focus areas, values, geographic scope."""
        
        try:
            # Use search-grounded call (Day 2: Tools)
            research = call_llm_with_search(AGENT_PROMPTS["company_researcher"], prompt)
            results["company_info"] = {"name": company_name, "research": research}
            company_memory.save_company(company_name, results["company_info"])
        except Exception as e:
            print(f"   ⚠️ Company research error: {e}")
            results["company_info"] = {"name": company_name, "research": "Research unavailable"}
    
    # Save CV to user memory (Day 3: Memory)
    user_memory.save_cv(user_id, cv_text, results["cv_data"])
    
    print("="*60)
    print("✅ PARALLEL PHASE COMPLETE")
    print("="*60 + "\n")
    
    return results


print("✅ Parallel analysis function ready (CLEANED)")
print("   - Single unified implementation")
print("   - Uses ADK agents via execute_adk_agent()")
print("   - ThreadPoolExecutor for concurrent execution")


## Cell 9: Panel-of-Experts Evaluation (ADK SequentialAgent Pattern)

**4 specialist evaluators** running in sequence, followed by aggregation and quality checks.

In [ ]:
# ============================================================================
# PANEL-OF-EXPERTS EVALUATION - ADK SequentialAgent Pattern (CLEANED)
# ============================================================================

EVALUATOR_AGENTS = [
    ("technical", technical_evaluator_agent),
    ("domain", domain_evaluator_agent),
    ("seniority", seniority_evaluator_agent),
    ("language", language_evaluator_agent)
]

def run_panel_of_experts(cv_data: Dict, job_data: Dict, 
                         cv_text: str = "", job_text: str = "") -> Dict:
    """
    Run the Panel-of-Experts evaluation using ADK SequentialAgent pattern.
    """
    print("\n" + "="*60)
    print("🎓 ADK SEQUENTIAL AGENT PHASE - Panel-of-Experts")
    print("   Evaluators: technical → domain → seniority → language")
    print("="*60)
    
    evaluations = {}
    
    # Build context for evaluators - FIXED: include language proficiency
    languages_detailed = cv_data.get('languages_detailed', [])
    languages_str = ", ".join([
        f"{lang.get('language', 'unknown')} (level {lang.get('proficiency', 3)}/5)"
        for lang in languages_detailed
    ]) if languages_detailed else "Not specified"
    
    cv_summary = f"""
Skills: {', '.join(cv_data.get('skills_case_sensitive', [])[:15])}
Expertise: {', '.join(cv_data.get('expertise_areas', [])[:10])}
Languages with proficiency: {languages_str}
Seniority: {cv_data.get('seniority_level', 'unknown')}
"""
    
    job_summary = f"""
Required skills: {', '.join(job_data.get('extracted_keywords', [])[:15])}
Required languages: {', '.join(job_data.get('required_languages', []))}
Preferred languages: {', '.join(job_data.get('preferred_languages', []))}
Seniority needed: {job_data.get('seniority_required', 'unknown')}
"""
    
    # Special context for language evaluator
    language_context = f"""
CANDIDATE LANGUAGES (with proficiency 1-5 scale):
{json.dumps(languages_detailed, indent=2)}

JOB LANGUAGE REQUIREMENTS:
- Required: {job_data.get('required_languages', [])}
- Preferred: {job_data.get('preferred_languages', [])}

SCORING GUIDE:
- Proficiency 5 (Excellent/Native) on required language = 100% credit
- Proficiency 4 (Advanced) on required language = 90% credit
- Proficiency 3 (Professional) on required language = 80% credit
- If candidate has required language at level 5 and no other requirements, score should be 90-100
"""
    
    # Run each evaluator in sequence
    for dimension, agent in EVALUATOR_AGENTS:
        print(f"   🔄 {dimension}_fit_evaluator running...")
        
        # Use special context for language evaluator
        if dimension == "language":
            prompt = f"""Evaluate this candidate for LANGUAGE FIT.

{language_context}

IMPORTANT: 
- If candidate has ALL required languages at proficiency 4-5, score should be 85-100
- If candidate has ALL required languages at proficiency 5, score should be 95-100
- Only penalize if required languages are MISSING or at low proficiency (1-2)

Return ONLY valid JSON with your evaluation including: dimension, score, strengths, gaps, notes."""
        else:
            prompt = f"""Evaluate this candidate for {dimension.upper()} FIT.

CANDIDATE PROFILE:
{cv_summary}

JOB REQUIREMENTS:
{job_summary}

Return ONLY valid JSON with your evaluation."""
        
        try:
            response = execute_adk_agent(agent, prompt)
            response = clean_json_response(response)
            evaluation = json.loads(response)
            evaluation["dimension"] = dimension
        except Exception as e:
            print(f"   ⚠️ {dimension} evaluation error: {e}")
            evaluation = {"dimension": dimension, "score": 50, "strengths": [], "gaps": [], "notes": "Evaluation failed"}
        
        evaluations[dimension] = evaluation
        print(f"   ✅ {dimension.title()}: {evaluation.get('score', 0)}/100")
    
    # Run aggregator
    print("\n   🔄 fit_aggregator_agent synthesizing...")
    aggregation = call_aggregator_agent(evaluations, cv_data, job_data)
    print(f"   ✅ Overall Score: {aggregation.get('overall_score', 0):.1f}/100")
    
    print("="*60)
    print("✅ PANEL-OF-EXPERTS COMPLETE")
    print("="*60 + "\n")
    
    return aggregation

def call_aggregator_agent(evaluations: Dict, cv_data: Dict, job_data: Dict) -> Dict:
    """Call fit_aggregator_agent to synthesize evaluations."""
    prompt = f"""Aggregate these specialist evaluations into a final assessment.

EVALUATIONS:
- Technical: {json.dumps(evaluations.get('technical', {}), indent=2)}
- Domain: {json.dumps(evaluations.get('domain', {}), indent=2)}
- Seniority: {json.dumps(evaluations.get('seniority', {}), indent=2)}
- Language: {json.dumps(evaluations.get('language', {}), indent=2)}

Calculate weighted overall score:
- Technical: 40%, Domain: 15%, Language: 20%, Seniority: 25%

Return ONLY valid JSON with: overall_score, dimensions, weights, summary (strengths, gaps, recommendation)"""
    
    try:
        response = execute_adk_agent(fit_aggregator_agent, prompt)
        response = clean_json_response(response)
        result = json.loads(response)
        
        if "overall_score" not in result:
            # Calculate manually if needed
            tech = evaluations.get('technical', {}).get('score', 50)
            domain = evaluations.get('domain', {}).get('score', 50)
            lang = evaluations.get('language', {}).get('score', 50)
            sen = evaluations.get('seniority', {}).get('score', 50)
            result["overall_score"] = tech * 0.4 + domain * 0.15 + lang * 0.2 + sen * 0.25
        
        result["dimensions"] = evaluations
        return result
        
    except Exception as e:
        print(f"   ⚠️ Aggregator error: {e}")
        # Fallback calculation
        tech = evaluations.get('technical', {}).get('score', 50)
        domain = evaluations.get('domain', {}).get('score', 50)
        lang = evaluations.get('language', {}).get('score', 50)
        sen = evaluations.get('seniority', {}).get('score', 50)
        overall = tech * 0.4 + domain * 0.15 + lang * 0.2 + sen * 0.25
        
        all_strengths = []
        all_gaps = []
        for dim in evaluations.values():
            all_strengths.extend(dim.get('strengths', []))
            all_gaps.extend(dim.get('gaps', []))
        
        recommendation = "Strong fit – proceed" if overall >= 75 else \
                        "Moderate fit – consider applying" if overall >= 50 else \
                        "Weak fit – significant gaps"
        
        return {
            "overall_score": overall,
            "dimensions": evaluations,
            "weights": {"technical": 0.40, "domain": 0.15, "language": 0.20, "seniority": 0.25},
            "summary": {"strengths": all_strengths[:7], "gaps": all_gaps[:7], "recommendation": recommendation}
        }


def call_constraint_checker(cv_data: Dict, job_data: Dict, panel_evaluation: Dict) -> Dict:
    """Call constraint_checker_agent."""
    print("   🚦 constraint_checker_agent running...")
    
    prompt = f"""Review whether this candidate meets HARD REQUIREMENTS.

JOB HARD REQUIREMENTS:
- Required languages: {job_data.get('required_languages', [])}
- Seniority: {job_data.get('seniority_required', 'Not specified')}

CANDIDATE PROFILE:
- Languages: {cv_data.get('languages', [])}
- Seniority: {cv_data.get('seniority_level', 'Unknown')}

Overall score: {panel_evaluation.get('overall_score', 50)}

Determine: GO, BORDERLINE, or NO_GO. Return JSON with: go_no_go, failed_constraints, warnings"""
    
    try:
        response = execute_adk_agent(constraint_checker_agent, prompt)
        response = clean_json_response(response)
        result = json.loads(response)
        print(f"   ✅ Constraint Check: {result.get('go_no_go', 'GO')}")
        return result
    except Exception as e:
        print(f"   ⚠️ Constraint checker error: {e}")
        return {"go_no_go": "GO", "failed_constraints": [], "warnings": []}


def call_risk_checker(all_evaluations: Dict) -> Dict:
    """Call risk_checker_agent."""
    print("   🛡️ risk_checker_agent running...")
    
    prompt = f"""Review these evaluations for quality and bias issues.

EVALUATION PACKAGE:
{json.dumps(all_evaluations, indent=2)[:2500]}

Check for bias, inconsistencies, weak signals. Return JSON with: risk_level, issues, bias_check_passed"""
    
    try:
        response = execute_adk_agent(risk_checker_agent, prompt)
        response = clean_json_response(response)
        result = json.loads(response)
        print(f"   ✅ Risk Level: {result.get('risk_level', 'LOW')}")
        return result
    except Exception as e:
        print(f"   ⚠️ Risk checker error: {e}")
        return {"risk_level": "LOW", "issues": [], "bias_check_passed": True}


print("✅ Panel-of-experts evaluation ready (CLEANED)")


## Cell 10: A2A Protocol Demonstration (Day 5)

**Agent-to-Agent (A2A) Communication** - Demonstrating how JobFit AI could integrate with external agent systems.

The A2A protocol enables agents to communicate across systems, enabling scenarios like:
- JobFit AI requesting company data from a Company Intelligence Agent
- External ATS agents requesting fit scores from JobFit AI
- HR agents delegating candidate evaluation to JobFit AI

In [ ]:
# ============================================================================
# A2A PROTOCOL DEMONSTRATION (Day 5: Prototype to Production)
# 
# The A2A (Agent-to-Agent) protocol enables inter-agent communication.
# This demonstrates how JobFit AI could expose its capabilities to other agents.
# ============================================================================

class JobFitA2AInterface:
    """
    A2A-style interface for JobFit AI.
    
    In a production deployment, this would be exposed via:
    - REST API endpoints
    - gRPC services  
    - ADK's native A2A protocol handlers
    
    This demonstrates the PATTERN for A2A integration.
    """
    
    def __init__(self):
        self.agent_id = "jobfit_ai_agent"
        self.version = "1.0.0"
        self.capabilities = [
            "cv_parsing",
            "job_parsing", 
            "fit_evaluation",
            "resume_generation",
            "cover_letter_generation"
        ]
    
    def get_agent_card(self) -> Dict:
        """
        Return A2A Agent Card - describes this agent's capabilities.
        
        In A2A protocol, agents advertise their capabilities via Agent Cards,
        allowing other agents to discover and invoke their services.
        """
        return {
            "agent_id": self.agent_id,
            "name": "JobFit AI",
            "description": "Intelligent job application evaluation using Panel-of-Experts architecture",
            "version": self.version,
            "capabilities": self.capabilities,
            "input_schema": {
                "cv_text": "string - Full text of candidate CV",
                "job_text": "string - Full text of job description",
                "user_id": "string - Optional user identifier for memory"
            },
            "output_schema": {
                "overall_score": "float - 0-100 fit score",
                "recommendation": "string - Action recommendation",
                "strengths": "list - Key matching points",
                "gaps": "list - Areas needing improvement"
            },
            "endpoints": {
                "evaluate_fit": "/a2a/evaluate",
                "parse_cv": "/a2a/parse/cv",
                "parse_job": "/a2a/parse/job",
                "generate_resume": "/a2a/generate/resume"
            }
        }
    
    def handle_a2a_request(self, request: Dict) -> Dict:
        """
        Handle incoming A2A request from another agent.
        
        Request format (A2A standard):
        {
            "action": "evaluate_fit",
            "sender_agent": "hr_assistant_agent",
            "payload": {
                "cv_text": "...",
                "job_text": "..."
            },
            "correlation_id": "uuid-123"
        }
        """
        action = request.get("action")
        payload = request.get("payload", {})
        correlation_id = request.get("correlation_id", "unknown")
        sender = request.get("sender_agent", "unknown")
        
        print(f"\n📨 A2A Request received from: {sender}")
        print(f"   Action: {action}")
        print(f"   Correlation ID: {correlation_id}")
        
        response = {
            "responder_agent": self.agent_id,
            "correlation_id": correlation_id,
            "status": "success"
        }
        
        try:
            if action == "evaluate_fit":
                # Run full evaluation
                cv_text = payload.get("cv_text", "")
                job_text = payload.get("job_text", "")
                
                if not cv_text or not job_text:
                    response["status"] = "error"
                    response["error"] = "Missing cv_text or job_text in payload"
                else:
                    # Use our existing pipeline
                    parallel_results = run_parallel_analysis(cv_text, job_text)
                    panel_results = run_panel_of_experts(
                        parallel_results["cv_data"],
                        parallel_results["job_data"],
                        cv_text, job_text
                    )
                    
                    response["result"] = {
                        "overall_score": panel_results.get("overall_score", 0),
                        "recommendation": panel_results.get("summary", {}).get("recommendation", ""),
                        "strengths": panel_results.get("summary", {}).get("strengths", []),
                        "gaps": panel_results.get("summary", {}).get("gaps", [])
                    }
            
            elif action == "get_capabilities":
                response["result"] = self.get_agent_card()
            
            elif action == "health_check":
                response["result"] = {"status": "healthy", "version": self.version}
            
            else:
                response["status"] = "error"
                response["error"] = f"Unknown action: {action}"
        
        except Exception as e:
            response["status"] = "error"
            response["error"] = str(e)
        
        print(f"   Response status: {response['status']}")
        return response
    
    def send_a2a_request(self, target_agent: str, action: str, payload: Dict) -> Dict:
        """
        Send A2A request to another agent (demonstration).
        
        In production, this would use HTTP/gRPC to communicate with remote agents.
        """
        import uuid
        
        request = {
            "action": action,
            "sender_agent": self.agent_id,
            "payload": payload,
            "correlation_id": str(uuid.uuid4())
        }
        
        print(f"\n📤 A2A Request to: {target_agent}")
        print(f"   Action: {action}")
        
        # In production, this would make an actual network call
        # For demo, we simulate a response
        return {
            "status": "simulated",
            "message": f"Would send to {target_agent}: {action}",
            "request": request
        }


# Initialize A2A interface
jobfit_a2a = JobFitA2AInterface()

# Demonstrate A2A capabilities
def demo_a2a_protocol():
    """Demonstrate A2A protocol integration."""
    print("\n" + "="*60)
    print("🔗 A2A PROTOCOL DEMONSTRATION (Day 5)")
    print("="*60)
    
    # 1. Get Agent Card
    print("\n1️⃣ Agent Card (capability advertisement):")
    card = jobfit_a2a.get_agent_card()
    print(f"   Agent: {card['name']}")
    print(f"   Capabilities: {', '.join(card['capabilities'])}")
    
    # 2. Simulate incoming request
    print("\n2️⃣ Simulated incoming A2A request:")
    sample_request = {
        "action": "get_capabilities",
        "sender_agent": "hr_recruitment_bot",
        "payload": {},
        "correlation_id": "demo-123"
    }
    response = jobfit_a2a.handle_a2a_request(sample_request)
    print(f"   Response: {response['status']}")
    
    # 3. Simulate outgoing request
    print("\n3️⃣ Simulated outgoing A2A request:")
    outgoing = jobfit_a2a.send_a2a_request(
        target_agent="company_intelligence_agent",
        action="get_company_info",
        payload={"company_name": "Google"}
    )
    print(f"   Status: {outgoing['status']}")
    
    print("\n" + "="*60)
    print("✅ A2A DEMONSTRATION COMPLETE")
    print("="*60)
    print("\n💡 In production, JobFit AI could:")
    print("   - Receive evaluation requests from ATS systems")
    print("   - Query company databases via A2A")
    print("   - Integrate with HR workflow agents")
    
    return card

# Run demo
a2a_card = demo_a2a_protocol()

print("\n✅ A2A Protocol interface ready")
print("   - jobfit_a2a.get_agent_card(): Advertise capabilities")
print("   - jobfit_a2a.handle_a2a_request(): Process incoming requests")
print("   - jobfit_a2a.send_a2a_request(): Send to other agents")


## Cell 11: Sequential Analysis (Fit Scoring)

In [ ]:
# ============================================================================
# SEQUENTIAL AGENT - FIT ANALYSIS (CLEANED)
# ============================================================================

def calculate_fit_score(cv_data: Dict, job_data: Dict, 
                        cv_text: str = "", job_text: str = "") -> Dict[str, Any]:
    """
    Calculate comprehensive fit score using PANEL-OF-EXPERTS architecture.
    """
    print("\n" + "="*60)
    print("📊 SEQUENTIAL AGENT PHASE - Panel-of-Experts Evaluation")
    print("="*60)
    
    # Run Panel-of-Experts
    panel_result = run_panel_of_experts(cv_data, job_data, cv_text, job_text)
    
    # Run Quality Checks
    constraint_result = call_constraint_checker(cv_data, job_data, panel_result)
    risk_result = call_risk_checker({
        "panel_evaluation": panel_result,
        "constraints": constraint_result,
        "cv_data": cv_data,
        "job_data": job_data
    })
    
    # Extract scores
    overall_score = panel_result.get("overall_score", 50)
    dimensions = panel_result.get("dimensions", {})
    technical_eval = dimensions.get("technical", {})
    domain_eval = dimensions.get("domain", {})
    seniority_eval = dimensions.get("seniority", {})
    language_eval = dimensions.get("language", {})
    summary = panel_result.get("summary", {})
    
    # Build output structure
    score_data = {
        "overall_score": round(overall_score, 2),
        "keyword_matches": technical_eval.get("strengths", [])[:15],
        "missing_keywords": technical_eval.get("gaps", [])[:15],
        "expertise_matches": domain_eval.get("strengths", [])[:10],
        "expertise_missing": domain_eval.get("gaps", [])[:10],
        "language_match": language_eval.get("score", 50) >= 70,
        "missing_languages": language_eval.get("required_languages_missing", []),
        "seniority_match": seniority_eval.get("score", 50) >= 70,
        "match_details": {
            "technical_score": round(technical_eval.get("score", 0) * 0.40, 2),
            "technical_max": 40,
            "expertise_score": round(domain_eval.get("score", 0) * 0.15, 2),
            "expertise_max": 15,
            "language_score": round(language_eval.get("score", 0) * 0.20, 2),
            "language_max": 20,
            "seniority_score": round(seniority_eval.get("score", 0) * 0.25, 2),
            "seniority_max": 25,
        },
        "panel_evaluation": panel_result,
        "constraint_check": constraint_result,
        "risk_assessment": risk_result,
        "qualitative_summary": {
            "strengths": summary.get("strengths", []),
            "gaps": summary.get("gaps", []),
            "recommendation": summary.get("recommendation", "Consider applying"),
            "go_no_go": constraint_result.get("go_no_go", "GO"),
            "risk_level": risk_result.get("risk_level", "LOW")
        }
    }
    
    print(f" 📈 Overall Score: {overall_score:.1f}%")
    print(f" 🚦 Constraint Check: {constraint_result.get('go_no_go', 'GO')}")
    print(f" 🛡️ Risk Level: {risk_result.get('risk_level', 'LOW')}")
    print(f" 💡 Recommendation: {summary.get('recommendation', 'N/A')}")
    
    print("\n" + "="*60)
    print("✅ SEQUENTIAL AGENT PHASE COMPLETE")
    print("="*60 + "\n")
    
    return score_data


def create_job_summary(job_data: Dict, company_name: str, location: str) -> Dict[str, Any]:
    """Create comprehensive job summary."""
    return {
        "company": company_name,
        "location": location,
        "keywords": safe_list(job_data.get("extracted_keywords")),
        "required_languages": safe_list(job_data.get("required_languages")),
        "preferred_languages": safe_list(job_data.get("preferred_languages")),
        "seniority": safe_string(job_data.get("seniority_required"), "Not specified"),
        "expertise_required": safe_list(job_data.get("expertise_areas_required")),
        "total_requirements": len(safe_list(job_data.get("extracted_keywords")))
    }


print("✅ Sequential analysis (fit scoring) ready")


## Cell 12: Content Generation (Resume & Cover Letter)

In [ ]:
# ============================================================================
# Cell 12: Content Generation (Resume & Cover Letter) - COMPLETE CORRECTED
# ============================================================================

from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from datetime import datetime
import re
import os


def generate_tailored_resume(cv_data: Dict, job_data: Dict, company_name: str, 
                             candidate_info: Dict, cv_text: str, job_text: str) -> str:
    """Generate ATS-optimized resume using ADK resume_tailor_agent."""
    cv_excerpt = cv_text[:3000] if len(cv_text) > 3000 else cv_text
    job_excerpt = job_text[:2000] if len(job_text) > 2000 else job_text
    
    prompt = f"""Generate a tailored, ATS-optimized resume for this candidate applying to {company_name}.

CANDIDATE CV EXCERPT:
{cv_excerpt}

JOB DESCRIPTION EXCERPT:
{job_excerpt}

CANDIDATE INFO:
- Name: {candidate_info.get('name', 'Candidate')}
- Email: {candidate_info.get('email', 'email@example.com')}
- Phone: {candidate_info.get('phone', '')}
- Location: {candidate_info.get('location', '')}

KEY SKILLS: {', '.join(cv_data.get('skills_case_sensitive', [])[:20])}

Create a professional resume that:
1. Uses ACTUAL candidate information
2. Highlights relevant experience
3. Is ATS-optimized with keywords from the job description

Return ONLY the formatted resume text."""
    
    try:
        print("   ✍️ resume_tailor_agent generating...")
        resume_text = execute_adk_agent(resume_tailor_agent, prompt)
        print("   ✅ Resume generated")
        return resume_text
    except Exception as e:
        return f"Error generating resume: {str(e)}"


def generate_cover_letter(cv_data: Dict, job_data: Dict, company_name: str,
                          company_info: Dict, candidate_info: Dict, 
                          fit_analysis: Dict) -> str:
    """Generate personalized cover letter using ADK cover_letter_agent."""
    strengths = fit_analysis.get('keyword_matches', [])[:5]
    overall_score = fit_analysis.get('overall_score', 0)
    
    prompt = f"""Write a compelling cover letter for {candidate_info.get('name', 'the candidate')} applying to {company_name}.

CANDIDATE:
- Name: {candidate_info.get('name', 'Candidate')}
- Email: {candidate_info.get('email', 'email@example.com')}
- Background: {cv_data.get('seniority_level', 'Professional')} with expertise in {', '.join(cv_data.get('expertise_areas', [])[:3])}

COMPANY: {company_info.get('research', 'Leading organization')[:500]}

JOB: {job_data.get('seniority_required', 'Professional level')}

STRENGTHS: {', '.join(strengths)}
FIT: {overall_score:.0f}%

REQUIREMENTS:
1. Use ACTUAL candidate name and email
2. Use TODAY'S date: {datetime.now().strftime('%B %d, %Y')}
3. Sound HUMAN - professional but conversational
4. 3-4 paragraphs, 350-450 words

Return ONLY the complete cover letter."""
    
    try:
        print("   ✉️ cover_letter_agent generating...")
        cover_letter = execute_adk_agent(cover_letter_agent, prompt)
        print("   ✅ Cover letter generated")
        return cover_letter
    except Exception as e:
        return f"Error generating cover letter: {str(e)}"


def parse_markdown_to_docx(doc, text: str):
    """Parse markdown text and add properly formatted content to a DOCX document."""
    from docx.shared import Pt
    from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
    
    lines = text.split('\n')
    
    for line in lines:
        stripped = line.strip()
        if not stripped:
            continue
        
        # Handle headers
        if stripped.startswith('# '):
            p = doc.add_paragraph()
            run = p.add_run(stripped[2:])
            run.bold = True
            run.font.size = Pt(16)
        elif stripped.startswith('## '):
            p = doc.add_paragraph()
            run = p.add_run(stripped[3:])
            run.bold = True
            run.font.size = Pt(14)
        elif stripped.startswith('### '):
            p = doc.add_paragraph()
            run = p.add_run(stripped[4:])
            run.bold = True
            run.font.size = Pt(12)
        elif stripped.startswith('- ') or stripped.startswith('* '):
            doc.add_paragraph(stripped[2:], style='List Bullet')
        elif stripped.startswith('**') and stripped.endswith('**'):
            p = doc.add_paragraph()
            run = p.add_run(stripped[2:-2])
            run.bold = True
        else:
            doc.add_paragraph(stripped)


def create_resume_docx(resume_text: str, candidate_name: str) -> str:
    """Create a formatted DOCX resume."""
    doc = Document()
    
    # Set narrow margins
    for section in doc.sections:
        section.top_margin = Inches(0.5)
        section.bottom_margin = Inches(0.5)
        section.left_margin = Inches(0.75)
        section.right_margin = Inches(0.75)
    
    parse_markdown_to_docx(doc, resume_text)
    
    safe_name = re.sub(r'[^a-zA-Z0-9]', '_', candidate_name)
    filename = f"Resume_{safe_name}_{datetime.now().strftime('%Y%m%d')}.docx"
    doc.save(filename)
    print(f"   📄 Resume saved: {filename}")
    
    # FIX: Return absolute path for Gradio download to work
    return os.path.abspath(filename)


def create_cover_letter_docx(cover_letter_text: str, candidate_name: str) -> str:
    """Create a formatted DOCX cover letter."""
    doc = Document()
    
    for section in doc.sections:
        section.top_margin = Inches(1)
        section.bottom_margin = Inches(1)
        section.left_margin = Inches(1)
        section.right_margin = Inches(1)
    
    paragraphs = cover_letter_text.split('\n\n')
    for para_text in paragraphs:
        if para_text.strip():
            doc.add_paragraph(para_text.strip())
    
    safe_name = re.sub(r'[^a-zA-Z0-9]', '_', candidate_name)
    filename = f"CoverLetter_{safe_name}_{datetime.now().strftime('%Y%m%d')}.docx"
    doc.save(filename)
    print(f"   📄 Cover letter saved: {filename}")
    
    # FIX: Return absolute path for Gradio download to work
    return os.path.abspath(filename)


print("✅ Content generation functions ready")

## Cell 13: JobFit Application Class

In [ ]:
# ============================================================================
# JOBFIT APPLICATION CLASS (CLEANED)
# ============================================================================

class JobFitApplication:
    """Main application class orchestrating all agents."""
    
    def __init__(self):
        self.memory = user_memory
        self.company_memory = company_memory
        self.last_analysis = None
    
    def process_application(self, user_id: str, job_text: str, cv_text: str) -> Dict[str, Any]:
        """Main entry point for processing a job application."""
        
        try:
            print("\n" + "="*70)
            print("🚀 JOBFIT AI - Starting Analysis")
            print("="*70)
            
            # PHASE 1: Parallel Analysis
            parallel_results = run_parallel_analysis(cv_text, job_text, user_id)
            
            cv_data = parallel_results["cv_data"]
            job_data = parallel_results["job_data"]
            company_name = parallel_results["company_name"]
            company_info = parallel_results["company_info"]
            candidate_info = parallel_results["candidate_info"]
            
            # PHASE 2: Sequential Evaluation
            fit_analysis = calculate_fit_score(cv_data, job_data, cv_text, job_text)
            
            overall_score = fit_analysis.get("overall_score", 0)
            qualitative_summary = fit_analysis.get("qualitative_summary", {})
            recommendation = qualitative_summary.get("recommendation", "Consider applying")
            
            # Create job summary
            job_summary = create_job_summary(
                job_data, 
                company_name,
                job_data.get("location", "Not specified")
            )
            
            # Store for lazy generation
            self.last_analysis = {
                "cv_data": cv_data,
                "job_data": job_data,
                "company_name": company_name,
                "company_info": company_info,
                "candidate_info": candidate_info,
                "fit_analysis": fit_analysis,
                "cv_text": cv_text,
                "job_text": job_text
            }
            
            # Record in history
            self.memory.add_application(
                user_id, company_name,
                job_data.get('seniority_required', 'Position'),
                {"fit_score": overall_score, "recommendation": recommendation}
            )
            
            return {
                "status": "success",
                "candidate_info": candidate_info,
                "cv_data": cv_data,
                "company_name": company_name,
                "company_info": company_info,
                "job_summary": job_summary,
                "job_data": job_data,
                "fit_analysis": fit_analysis,
                "qualitative_analysis": qualitative_summary,
                "overall_score": overall_score,
                "timestamp": datetime.now().isoformat()
            }
        
        except Exception as e:
            import traceback
            return {
                "status": "error",
                "error": str(e),
                "error_trace": traceback.format_exc()
            }
    
    def generate_resume_lazy(self) -> Tuple[str, str]:
        """Generate tailored resume on-demand."""
        if self.last_analysis is None:
            return "❌ No analysis found. Please analyze a job first.", None
        
        print("\n🔄 Generating tailored resume...")
        
        try:
            resume_text = generate_tailored_resume(
                cv_data=self.last_analysis["cv_data"],
                job_data=self.last_analysis["job_data"],
                company_name=self.last_analysis["company_name"],
                candidate_info=self.last_analysis["candidate_info"],
                cv_text=self.last_analysis["cv_text"],
                job_text=self.last_analysis["job_text"]
            )
            
            docx_path = create_resume_docx(
                resume_text,
                self.last_analysis["candidate_info"].get("name", "Candidate")
            )
            
            return f"# ✅ Tailored Resume Generated\n\n{resume_text[:500]}...", docx_path
            
        except Exception as e:
            import traceback
            return f"❌ Error: {str(e)}\n{traceback.format_exc()}", None
    
    def generate_cover_letter_lazy(self) -> Tuple[str, str]:
        """Generate cover letter on-demand."""
        if self.last_analysis is None:
            return "❌ No analysis found. Please analyze a job first.", None
        
        print("\n🔄 Generating cover letter...")
        
        try:
            cover_letter = generate_cover_letter(
                cv_data=self.last_analysis["cv_data"],
                job_data=self.last_analysis["job_data"],
                company_name=self.last_analysis["company_name"],
                company_info=self.last_analysis["company_info"],
                candidate_info=self.last_analysis["candidate_info"],
                fit_analysis=self.last_analysis["fit_analysis"]
            )
            
            docx_path = create_cover_letter_docx(
                cover_letter,
                self.last_analysis["candidate_info"].get("name", "Candidate")
            )
            
            return f"# ✅ Cover Letter Generated\n\n{cover_letter[:500]}...", docx_path
            
        except Exception as e:
            import traceback
            return f"❌ Error: {str(e)}\n{traceback.format_exc()}", None
    
    def get_user_history(self, user_id: str) -> Dict:
        """Get user's application history."""
        return self.memory.get_user_history(user_id)


# Initialize application
jobfit = JobFitApplication()

print("✅ JobFit AI application instance created: 'jobfit'")
print("   - jobfit.process_application(user_id, job_text, cv_text)")
print("   - jobfit.generate_resume_lazy()")
print("   - jobfit.generate_cover_letter_lazy()")


## Cell 14: File Processing Functions

In [ ]:
# ============================================================================
# FILE PROCESSING FUNCTIONS
# ============================================================================

def extract_text_from_pdf(pdf_file) -> str:
    """Extract text from uploaded PDF file."""
    try:
        text = ""
        with pdfplumber.open(pdf_file) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        return text.strip()
    except Exception as e:
        return f"Error reading PDF: {str(e)}"


def extract_text_from_docx(docx_file) -> str:
    """Extract text from uploaded DOCX file."""
    try:
        doc = Document(docx_file)
        text = [para.text for para in doc.paragraphs if para.text.strip()]
        return "\n".join(text)
    except Exception as e:
        return f"Error reading DOCX: {str(e)}"


def process_uploaded_cv(file) -> Optional[str]:
    """Process uploaded CV file (PDF or DOCX)."""
    if file is None:
        return None
    
    file_path = file.name
    file_extension = os.path.splitext(file_path)[1].lower()
    
    if file_extension == '.pdf':
        return extract_text_from_pdf(file_path)
    elif file_extension in ['.docx', '.doc']:
        return extract_text_from_docx(file_path)
    else:
        return f"Unsupported file format: {file_extension}"


def fetch_google_doc_content(doc_url: str) -> Tuple[str, bool, str]:
    """Fetch text content from a Google Docs sharing link."""
    try:
        doc_id_match = re.search(r'/d/([a-zA-Z0-9-_]+)', doc_url)
        if not doc_id_match:
            return "", False, "Invalid Google Docs URL format"
        
        doc_id = doc_id_match.group(1)
        export_url = f"https://docs.google.com/document/d/{doc_id}/export?format=txt"
        
        response = requests.get(export_url, timeout=10)
        response.raise_for_status()
        
        content = response.text.strip()
        if not content:
            return "", False, "Document appears empty or not accessible"
        
        return content, True, ""
        
    except Exception as e:
        return "", False, f"Error fetching Google Docs: {str(e)}"


print("✅ File processing functions ready")


## Cell 15: Gradio Interface Functions

In [ ]:
# ============================================================================
# GRADIO INTERFACE FUNCTIONS (CORRECTED v2)
# ============================================================================

def process_cv_input(cv_file, cv_url: str, user_id: str) -> Tuple[str, str, bool]:
    """Process CV from file upload, URL, or memory."""
    
    # Priority 1: File upload
    if cv_file is not None:
        cv_text = process_uploaded_cv(cv_file)
        if cv_text:
            return cv_text, "uploaded file", True
        return "❌ Could not extract text from file", "error", False
    
    # Priority 2: Google Docs URL
    if cv_url and cv_url.strip():
        content, success, error_msg = fetch_google_doc_content(cv_url.strip())
        if success:
            return content, "Google Docs link", True
        return f"❌ {error_msg}", "error", False
    
    # Priority 3: Memory
    if user_id and user_memory.has_cv(user_id.strip()):
        stored_cv = user_memory.get_cv(user_id.strip())
        if stored_cv and stored_cv.get("text"):
            return stored_cv["text"], "memory", True
    
    return "", "none", False


def process_job_input(job_text: str, job_url: str) -> Tuple[str, str, bool]:
    """Process job description from text or URL."""
    if job_text and job_text.strip():
        return job_text.strip(), "text area", True
    
    if job_url and job_url.strip():
        content, success, error_msg = fetch_google_doc_content(job_url.strip())
        if success:
            return content, "Google Docs link", True
        return f"❌ {error_msg}", "error", False
    
    return "", "none", False


def process_job_application_ui(user_id, cv_file, cv_url, job_description, job_url):
    """Main UI processing function for Gradio interface."""
    
    if not user_id or not user_id.strip():
        yield ("❌ Please provide your email/user ID", "", "", "", "", "")
        return
    
    cv_text, cv_source, cv_success = process_cv_input(cv_file, cv_url, user_id)
    
    if not cv_success and cv_source == "none":
        yield ("❌ Please upload a CV file or provide a Google Docs link", "", "", "", "", "")
        return
    
    if not cv_success:
        yield (cv_text, "", "", "", "", "")
        return
    
    job_text, job_source, job_success = process_job_input(job_description, job_url)
    
    if not job_success:
        if job_source == "none":
            yield ("❌ Please paste the job description or provide a Google Docs link", "", "", "", "", "")
            return
        yield (job_text, "", "", "", "", "")
        return
    
    # Show loading message in Summary tab while processing
    yield (
        "## ⏳ Analyzing...\n\nRunning 14-agent pipeline. This typically takes 30-45 seconds.\n\n"
        "- 🔄 Parsing CV...\n"
        "- 🔄 Parsing Job Description...\n"
        "- 🔄 Researching Company...\n"
        "- 🔄 Running Expert Panel Evaluation...\n"
        "- 🔄 Checking Constraints...\n"
        "- 🔄 Running Risk Analysis...",
        "⏳ Loading...",
        "⏳ Loading...",
        "⏳ Loading...",
        "⏳ Loading...",
        "⏳ Loading..."
    )
    
    try:
        # Run analysis
        result = jobfit.process_application(user_id.strip(), job_text, cv_text)
        
        if result.get("status") == "error":
            yield (f"❌ Error: {result.get('error')}", "", "", "", "", "")
            return
        
        # Format outputs
        overall_score = result.get("overall_score", 0)
        qual = result.get("qualitative_analysis", {})
        
        summary = f"""# 📊 Fit Analysis Results

## Overall Score: {overall_score:.1f}/100

**Recommendation**: {qual.get('recommendation', 'N/A')}
**Go/No-Go**: {qual.get('go_no_go', 'GO')}
**Risk Level**: {qual.get('risk_level', 'LOW')}

### Key Strengths
{chr(10).join('• ' + s for s in qual.get('strengths', [])[:5])}

### Areas to Address  
{chr(10).join('• ' + g for g in qual.get('gaps', [])[:5])}
"""
        
        job_summary = result.get("job_summary", {})
        vacancy = f"""# 📋 Job Analysis

**Company**: {job_summary.get('company', 'Unknown')}
**Location**: {job_summary.get('location', 'Not specified')}
**Seniority**: {job_summary.get('seniority', 'Not specified')}

**Required Languages**: {', '.join(job_summary.get('required_languages', ['None specified']))}
**Key Requirements**: {len(job_summary.get('keywords', []))} identified
"""
        
        company_info = result.get("company_info", {})
        company = f"""# 🏢 Company Research

**{company_info.get('name', 'Unknown Company')}**

{company_info.get('research', 'No research available')[:1500]}
"""
        
        candidate = result.get("candidate_info", {})
        cv_data = result.get("cv_data", {})
        cv_profile = f"""# 👤 Candidate Profile

**Name**: {candidate.get('name', 'Unknown')}
**Email**: {candidate.get('email', 'Not provided')}
**Location**: {candidate.get('location', 'Not specified')}
**Seniority**: {cv_data.get('seniority_level', 'Unknown')}

**Languages**: {', '.join(cv_data.get('languages', ['Not specified']))}
**Key Skills**: {', '.join(cv_data.get('skills_case_sensitive', [])[:10])}
"""
        
        fit = result.get("fit_analysis", {})
        details = fit.get("match_details", {})
        fit_analysis_display = f"""# 📈 Detailed Fit Analysis

## Score Breakdown

| Dimension | Score | Max |
|-----------|-------|-----|
| Technical | {details.get('technical_score', 0):.1f} | 40 |
| Domain | {details.get('expertise_score', 0):.1f} | 15 |
| Language | {details.get('language_score', 0):.1f} | 20 |
| Seniority | {details.get('seniority_score', 0):.1f} | 25 |
| **TOTAL** | **{overall_score:.1f}** | **100** |
"""
        
        # History
        history = jobfit.get_user_history(user_id.strip())
        history_text = f"""# 📜 Application History

**Total Applications**: {history.get('application_count', 0)}
**CV on File**: {'Yes' if history.get('has_cv') else 'No'}
"""
        
        yield (summary, vacancy, company, cv_profile, fit_analysis_display, history_text)
        
    except Exception as e:
        import traceback
        yield (f"❌ Error: {str(e)}\n{traceback.format_exc()}", "", "", "", "", "")


def generate_resume_ui():
    """Generate resume and return downloadable file."""
    resume_text, resume_path = jobfit.generate_resume_lazy()
    if resume_path and os.path.exists(resume_path):
        # FIX: Return the path directly, not gr.update()
        return resume_text, resume_path
    return resume_text, None


def generate_cover_letter_ui():
    """Generate cover letter and return downloadable file."""
    cover_text, cover_path = jobfit.generate_cover_letter_lazy()
    if cover_path and os.path.exists(cover_path):
        # FIX: Return the path directly, not gr.update()
        return cover_text, cover_path
    return cover_text, None


print("✅ Gradio interface functions ready")

## Cell 16: Gradio Interface Launch

In [ ]:
# ============================================================================
# Cell 16: Gradio Interface Launch (CORRECTED v2)
# ============================================================================

import gradio as gr

# Demo links
DEMO_CV_URL = "https://docs.google.com/document/d/1NLhLMGFbtnwYgDHqpPrspZxa_fVxZZF6/edit?usp=sharing"
DEMO_JOB_URL = "https://docs.google.com/document/d/1W40CV4dkoskk1owSmfGUeyLQrtMqxHaZ/edit?usp=drive_link"


with gr.Blocks(title="JobFit AI - Panel-of-Experts v6") as demo:
    
    gr.Markdown("""
    # 🎯 JobFit AI - Intelligent Job Application Assistant
    
    ### ADK Agent Architecture (14 Agents)
    **ParallelAgent**: CV/Job/Company parsing (concurrent) |  **SequentialAgent**: Expert panel evaluation chain | **Quality Agents**: Constraint checker, Risk checker | **Tools**: Google Search for company research
    
    **Model**: Gemini 2.5 Flash Lite | **Runtime**: ~30-45 seconds
                
    ---
    """)
    
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### Step 1: Your Email")
            user_email = gr.Textbox(
                placeholder="your.email@example.com",
                value="demo@example.com",
                show_label=False
            )
            
            gr.Markdown("### Step 2: Your CV")
            gr.Markdown("*Demo link pre-filled*")
            cv_file = gr.File(
                label="Upload CV (PDF/DOCX)",
                file_types=[".pdf", ".docx"],
                type="filepath",
                height=120
            )
            cv_url = gr.Textbox(
                placeholder="Google Docs link",
                value=DEMO_CV_URL,
                show_label=False
            )
            
            gr.Markdown("### Step 3: Job Description")
            gr.Markdown("*Demo link pre-filled*")
            job_description = gr.Textbox(
                placeholder="Paste job description here...",
                lines=3,
                show_label=False
            )
            job_url = gr.Textbox(
                placeholder="Google Docs link",
                value=DEMO_JOB_URL,
                show_label=False
            )
            
            gr.Markdown("### Step 4: Analyze")
            analyze_btn = gr.Button("🔍 Analyze Fit", variant="primary", size="lg")
            
            gr.Markdown("### Step 5: Generate (Optional)")
            with gr.Row():
                resume_btn = gr.Button("📄 Generate Resume")
                cover_btn = gr.Button("✉️ Generate Cover Letter")
        
        with gr.Column(scale=2):
            with gr.Tabs():
                with gr.Tab("📊 Summary"):
                    summary_output = gr.Markdown()
                with gr.Tab("📋 Job"):
                    vacancy_output = gr.Markdown()
                with gr.Tab("🏢 Company"):
                    company_output = gr.Markdown()
                with gr.Tab("👤 Profile"):
                    cv_profile_output = gr.Markdown()
                with gr.Tab("📈 Fit Details"):
                    fit_analysis_output = gr.Markdown()
                with gr.Tab("📜 History"):
                    history_output = gr.Markdown()
                with gr.Tab("📄 Resume"):
                    resume_output = gr.Markdown()
                    # FIX: Use gr.File with interactive=False for download-only behavior
                    resume_download = gr.File(
                        label="📥 Download Resume", 
                        interactive=False,  # Makes it download-only
                        visible=True        # Always visible, shows "No file" when empty
                    )
                with gr.Tab("✉️ Cover Letter"):
                    cover_output = gr.Markdown()
                    # FIX: Use gr.File with interactive=False for download-only behavior
                    cover_download = gr.File(
                        label="📥 Download Cover Letter", 
                        interactive=False,  # Makes it download-only
                        visible=True        # Always visible, shows "No file" when empty
                    )
    
    # Event handlers
    analyze_btn.click(
        fn=process_job_application_ui,
        inputs=[user_email, cv_file, cv_url, job_description, job_url],
        outputs=[summary_output, vacancy_output, company_output, cv_profile_output, 
                 fit_analysis_output, history_output],
        show_progress='full'
    )

    resume_btn.click(
        fn=generate_resume_ui,
        inputs=[],
        outputs=[resume_output, resume_download],
        show_progress='full'
    )
    
    cover_btn.click(
        fn=generate_cover_letter_ui,
        inputs=[],
        outputs=[cover_output, cover_download],
        show_progress='full'
    )


# ============================================================================
# LAUNCH - Different behavior for Kaggle vs Local
# ============================================================================

if "KAGGLE_KERNEL_RUN_TYPE" in os.environ:
    # Kaggle environment
    print("🚀 Launching Gradio in Kaggle mode...")
    demo.launch(share=False, debug=False)
else:
    # Local Jupyter environment
    print("🚀 Launching Gradio in local mode...")
    demo.launch(theme=gr.themes.Soft(), share=False, debug=True, show_error=True)

print("✅ Gradio interface launched")

## Limitations & Future Work

**Current Limitations**
- **Model dependence**: Quality depends on underlying Gemini model and input clarity
- **Domain bias**: Prompts tuned for international organizations; other sectors may need adjustment
- **Session-only memory**: UserMemory and CompanyMemory are in-memory only; no persistence across restarts
- **No ATS integration**: Does not fetch live job listings or submit to ATS platforms

**Planned Extensions**
- **Sector-specific profiles**: Add evaluation profiles for tech, public sector, etc.
- **Persistent storage**: Connect memory to database/vector store
- **A2A Integration**: Full A2A protocol implementation for inter-agent communication
- **Iterative refinement**: Loop-style agents for multiple passes on documents

---

## 📚 Course Concepts Summary

| Day | Concept | Implementation |
|-----|---------|----------------|
| 1 | Agent Architectures | 14 LlmAgents, ParallelAgent, SequentialAgent |
| 2 | Tools & MCP | google_search tool for company research |
| 3 | Memory | UserMemory, CompanyMemory services |
| 4 | Quality/Evaluation | LoggingPlugin, constraint_checker, risk_checker |
| 5 | Production | Runner, InMemorySessionService, A2A demo |

**Total capabilities demonstrated: 5+ (exceeds minimum of 3)**
